## Лабораторная работа №1

Седунов Илья,<br>
Альперович Вадим,<br>
17ПМИ

### Целью данной работы является сравнение следующих подходов:

1) **Квадратичный подход** к построению идеальной хеш-таблицы; (см. Кормен, 3-е издание,
11.5 Идеальное хеширование, или видео запись)

2) **Двух-уровневый подход** к построению идеальной хеш-таблицы; (см. Кормен, 3-е издание,
11.5 Идеальное хеширование, или видео запись)

3) (опционально) **Графовый подход** к построению идеальной хеш-таблицы
(см. https://habrahabr.ru/post/254431/, вот исходная статья, обязательно посмотрите
http://cmph.sourceforge.net/papers/chm92.pdf). 

### Что измеряется?

1) Время построения хеш-таблицы;

2) Время операции поиска элемента;

3) Затрачиваемая память

**Замечание:** В качестве хеш-функций используйте только функции из универсальных
семейств, про них рассказывалось на семинарах. Список универсальных хеш-функций
можно найти здесь https://en.wikipedia.org/wiki/Universal_hashing.
    
**Замечание:** Для двух-уровневой хеш-таблицы нужно обязательно провести эксперименты с
разлиными значениями параметров, разобраться как различные значения параметров влияют
на производительность и найти оптимальный выбор параметров. Без этих экспериментов
работа не будет зачтена. Про какие именно параметры идет речь смотрите в записи лекции

### Входные данные:

a) Случайные натуральные числа.

б) Случайные вектора или строки.

с) Очень бы хотелось увидеть как поведут себя таблицы на real life данных, например на
словарях или словах какого нибудь литературного произведения.

### Ваш вывод должен содержать:

1) Графики сравнения скорости построения и количества занимаемой памяти для подхода 1)
при различных значениях параметров. Выбор оптимального, с Вашей точки зрения, набора
параметров.

2) График сравнения скорости построения хеш-таблиц для подходов 1) 2) 3), причем для
подхода 2) нужно использовать оптимальный набор параметров.

3) График сравнения скорости поиска для обоих подходов.

4) Все графики нужно продублировать, если Вы используете разные входные данные:
случайные строки и real life данные.

Итого, должно быть как минимум 4 картинки с несколькими кривыми на каждой.

In [1]:
import numpy as np
import pandas as pd